In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Row

# Used this link for connecting all spark notebook to postgres instance.

https://stackoverflow.com/questions/67062205/adding-postgressql-jdbc-driver-to-all-spark-notebook-using-docker-compose

In [2]:
spark = SparkSession.builder.config("spark.driver.extraClassPath", "/home/jovyan/.ivy2/jars/org.postgresql_postgresql-42.2.19.jar") \
	.master("local").appName("PySpark_Postgres_test").getOrCreate()

In [3]:
studentDf = spark.createDataFrame([
	Row(id=1,name='vijay',marks=67),
	Row(id=2,name='Ajay',marks=88),
	Row(id=3,name='jay',marks=79),
	Row(id=4,name='vinay',marks=67),
])

In [4]:
studentDf.show(5)

+---+-----+-----+
| id| name|marks|
+---+-----+-----+
|  1|vijay|   67|
|  2| Ajay|   88|
|  3|  jay|   79|
|  4|vinay|   67|
+---+-----+-----+



In [24]:
 studentDf.select("id","name","marks").write.format("jdbc",)\
    .option("url", "jdbc:postgresql://pgdatabase:5432/ny_taxi") \
    .option("driver", "org.postgresql.Driver").option("dbtable", "students") \
    .option("user", "root").option("password", "root").save() 

1. Import the required PySpark modules and create a PySpark session with the PostgreSQL JDBC driver

In [13]:
spark = SparkSession.builder.config("spark.driver.extraClassPath", "/home/jovyan/.ivy2/jars/org.postgresql_postgresql-42.2.19.jar") \
	.master("local").appName("PySpark_Postgres_test").getOrCreate()

2. Import the required PySpark modules and create a PySpark session with the PostgreSQL JDBC driver

In [14]:
url = "jdbc:postgresql://pgdatabase:5432/ny_taxi"

properties = {
    "user": "root",
    "password": "root",
    "driver": "org.postgresql.Driver"
}

3. Querying a PostgreSQL Table using PySpark

In [15]:
table_name = "students"

df = spark.read.jdbc(url, table_name, properties=properties)

In [16]:
df.show()

+---+-----+-----+
| id| name|marks|
+---+-----+-----+
|  1|vijay|   67|
|  2| Ajay|   88|
|  3|  jay|   79|
|  4|vinay|   67|
+---+-----+-----+



4. Perform more complex queries using SQL
   
If you prefer to write SQL queries, you can register the DataFrame as a temporary table and then use SQL to query the data.

Register the DataFrame as a temporary table and replace your_temp_table with a name for the temporary table

In [18]:
df.createOrReplaceTempView("your_temp_table")

sql_query = "SELECT * FROM your_temp_table WHERE marks > 68"

result_df = spark.sql(sql_query)

In [20]:
result_df.show()

+---+----+-----+
| id|name|marks|
+---+----+-----+
|  2|Ajay|   88|
|  3| jay|   79|
+---+----+-----+



5. Save the results back to PostgreSQL

In [26]:
result_table_name = "students"
df.write.jdbc(url, result_table_name, mode="append", properties=properties)